In [9]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from newsapi import NewsApiClient
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
analyzer = SentimentIntensityAnalyzer()

In [11]:
# Read your api key environment variable
# YOUR CODE HERE!
api_key = os.getenv("news_api")

In [13]:
# Create a newsapi client
# YOUR CODE HERE!
newsapi = NewsApiClient(api_key=api_key)

In [23]:
# Fetch the Bitcoin news articles
# YOUR CODE HERE!
btc_headlines = newsapi.get_everything(
    q="bitcoin",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Show sample article
btc_headlines["articles"][0]


{'source': {'id': 'wired', 'name': 'Wired'},
 'author': 'Khari Johnson',
 'title': 'Why Not Use Self-Driving Cars as Supercomputers?',
 'description': 'Autonomous vehicles use the equivalent of 200 laptops to get around. Some want to tap that computing power to decode viruses or mine bitcoin.',
 'url': 'https://www.wired.com/story/use-self-driving-cars-supercomputers/',
 'urlToImage': 'https://media.wired.com/photos/60f081b4c147fe7a1a367362/191:100/w_1280,c_limit/Business-Autonomous-Vehicles-Supercomputers-1201885684.jpg',
 'publishedAt': '2021-07-19T11:00:00Z',
 'content': 'Like Dogecoin devotees, the mayor of Reno, and the leaders of El Salvador, Aldo Baoicchi is convinced cryptocurrency is the future. The CEO and founder of Canadian scooter maker Daymak believes this … [+4116 chars]'}

In [24]:
# Create the Bitcoin sentiment scores DataFrame
# YOUR CODE HERE!
btc_sentiments = []

for article in btc_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(btc_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()

,date,text,compound,positive,negative,neutral
0,2021-07-19,"Like Dogecoin devotees, the mayor of Reno, and...",0.6908,0.178,0.000,0.822
1,2021-07-05,Filed under:\r\nThe supply chain attack has re...,-0.5719,0.111,0.184,0.705
2,2021-07-05,image copyrightGetty Images\r\nThe gang behind...,-0.6124,0.000,0.143,0.857
3,2021-07-23,To get a roundup of TechCrunchs biggest and mo...,0.6240,0.127,0.000,0.873
4,2021-07-14,While retail investors grew more comfortable b...,0.7264,0.164,0.000,0.836


In [25]:
btc_sorted = btc_df.sort_values(by=['date'])

In [26]:
btc_sorted.head()

,date,text,compound,positive,negative,neutral
73,2021-06-27,"By Nduka OrjinmoBBC News, Abuja\r\nimage copyr...",-0.6124,0.000,0.139,0.861
38,2021-06-28,The Fine Print: The following comments are own...,-0.0418,0.079,0.086,0.835
84,2021-06-28,This article was translated from our Spanish e...,-0.3400,0.000,0.068,0.932
20,2021-06-28,Representations of the Bitcoin cryptocurrency ...,0.0000,0.000,0.000,1.000
45,2021-06-28,"NEW YORK, June 28 (Reuters) - Ether investment...",-0.5719,0.000,0.101,0.899


In [30]:
# Describe the Bitcoin Sentiment
# YOUR CODE HERE!
btc_sorted.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.075597,0.055760,0.035160,0.909060
std,0.386608,0.066664,0.059152,0.085565
min,-0.827100,0.000000,0.000000,0.653000
25%,-0.051600,0.000000,0.000000,0.857000
50%,0.000000,0.047000,0.000000,0.915500
75%,0.345300,0.092000,0.067250,1.000000
max,0.883400,0.276000,0.287000,1.000000


In [29]:
btc_sorted.to_csv(r'C:\Users\lucas\Desktop\Fintech Bootcamp\Project#2\BTC_sentiment.csv', index = False)